In [ ]:
import cv2
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import random


In [ ]:
def load_calibration(calib_file):
    """Loads stereo calibration parameters from an .npz file."""
    with np.load(calib_file) as data:
        map1x = data['map1x']
        map1y = data['map1y']
        map2x = data['map2x']
        map2y = data['map2y']
        Q = data['Q']
    
    return map1x, map1y, map2x, map2y, Q

def rectify_stereo_pair(img0, img1, map1x, map1y, map2x, map2y):
    """
    Takes a new stereo pair (img0, img1), rectifies them using the provided maps,
    and returns the undistorted, rectified pair.
    """
    rect0 = cv2.remap(img0, map1x, map1y, cv2.INTER_LINEAR)
    rect1 = cv2.remap(img1, map2x, map2y, cv2.INTER_LINEAR)
    return rect0, rect1

In [ ]:
map1x, map1y, map2x, map2y, Q = load_calibration("calib.npz")

ir_images = [] # Right camera
rgb_images = [] # Left camera
directory = "./new_calib_data"
for filename in os.listdir(directory):
    if filename.endswith("_ir.jpg"):
        ir_images.append(os.path.join(directory, filename))
    elif filename.endswith("_rgb.jpg"):
        rgb_images.append(os.path.join(directory, filename))
ir_images.sort()
rgb_images.sort()
print("IR images: ", ir_images)
print("RGB images: ", rgb_images)
rand_indx = random.randint(0, len(ir_images)-1)
img0 = cv2.imread(ir_images[rand_indx])
img1 = cv2.imread(rgb_images[rand_indx])

rectified0, rectified1 = rectify_stereo_pair(img0, img1, map1x, map1y, map2x, map2y)

cv2.imwrite("rectified_ir.jpg", rectified0)
cv2.imwrite("rectified_rgb.jpg", rectified1)

In [ ]:
# Load rectified images
left_img = cv2.imread("rectified_rgb.jpg")
right_img = cv2.imread("rectified_ir.jpg")

# Convert to grayscale for easier visual comparison
gray_left = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
gray_right = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)

# Display images side by side
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(gray_left, cmap="gray")
plt.title("Rectified Left Image")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(gray_right, cmap="gray")
plt.title("Rectified Right Image")
plt.axis("off")

plt.show()

In [ ]:
def draw_epipolar_lines(img1, img2, num_lines=10):
    h, w = img1.shape[:2]
    step = h // num_lines  # Spacing between lines

    img1_lines = img1.copy()
    img2_lines = img2.copy()

    for i in range(0, h, step):
        cv2.line(img1_lines, (0, i), (w, i), (0, 255, 0), 1)
        cv2.line(img2_lines, (0, i), (w, i), (0, 255, 0), 1)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(img1_lines, cv2.COLOR_BGR2RGB))
    plt.title("Left Image with Epipolar Lines")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(img2_lines, cv2.COLOR_BGR2RGB))
    plt.title("Right Image with Epipolar Lines")
    plt.axis("off")

    plt.show()

# Load rectified images
left_img = cv2.imread("rectified_rgb.jpg")
right_img = cv2.imread("rectified_ir.jpg")

draw_epipolar_lines(left_img, right_img)


In [ ]:
stereo = cv2.StereoSGBM_create(
    minDisparity=0, numDisparities=64, blockSize=9
)
disparity = stereo.compute(gray_left, gray_right).astype(np.float32) / 16.0

plt.figure(figsize=(10, 6))
plt.imshow(disparity, cmap='jet')
plt.colorbar()
plt.title("Disparity Map")
plt.axis("off")
plt.show()